# Office Covid Simulation

This project simulates the spread of COVID-19 in the workplace. Through use of a GUI or a text file input, users are able to specify a set of initial parameters:

* Number of people – the number of people in the office, less than the number of desks available in the office plan. 

* Number of infected – the number of people that start the simulation infected. 

* Minimum and maximum age – the upper and lower limits of the possible age of people in the office. Age is used to calculate chance of death. 

* Mask adherence – the probability that an individual person is wearing a mask. 

* Social distancing adherence – the probability that a person takes a path that leaves at least 1m between themselves and another person.  

* Virality – a factor that increases the likelihood of COVID-19 transmission when an infectious person interacts with an uninfected person. 

* Floor number – corresponds to the excel file sheet number which contains floorplans that define the movable space for office workers. 

* Simulation duration – the number of discrete time steps the simulation is run for. 

## Systems Diagram

<img src="Report_graphics/systems_diagram.png" width=600 height=600 />

                                    Figure 1   Systems diagram showing the process followed.

## How to Run

The simulation can have parameters input either from a .txt file or GUI, by launching from run_covid_simulation.py with either a  --GUI flag or no flag but an input text file argument. The latter allows for further scripts to be written that could run multiple simulations in the background without graphical interruptions, to compare office set ups using many different parameter combinations. 

## GUI

The Graphical User Interface (GUI) was created using the python module Tkinter and features various widgets which can be controlled by the user to define the input parameters to the simulation. The widgets are organised into a grid format such that they maintain their relative positions as the window is resized. Figure xxx shows an image of the GUI. The following widgets are used:

<img src="Report_graphics/GUI_startup.png" width=600 height=600 />

Figure 2 GUI on startup

* Spin boxes (number of people and number of infected)
* Sliders (max age, min age, mask adherence, social distancing adherence, virality and simulation duration) 
* List box (floor number) 
* Buttons (begin simulation, replay animation, save animation, quit app)

Each widget utilises ‘command’ or ‘lamba’ call-back functions which are triggered when clicked on. These functions are used to update the input parameters and control the maximum value, minimum value and status of other widgets. This is best understood by considering the user changes:

* The number of people and number of infected are interlinked such that the number of people is always greater than or equal to the number of infected.
* The maximum age must always exceed the minimum age otherwise the begin simulation box is greyed out such that it cannot be pressed.
* The maximum number of people is defined by the number of desks (a feature of each floor)  – this means when the floor is changed the maximum value of the number of people is changed (if the number of people exceeds the number of desks when a floor change occurs then the number of people is set to the number of desks)
* The replay animation and save animation buttons can only be pressed when there is an animation to replay or save.


These controls aim to prevent the user from specifying invalid parameters which could break the simulation. 

## Office Class

The office class controls and displays the office space in which people move and interact. There are four arrays calculated by the office class:

* Input array: one input array is created when the simulation starts, by processing the floor plan selected from office_array.xls to identify walls (0), traversable space (1), tasks (T) and desks (D).
* Pathfinding arrays: multiple pathfinding arrays are calculated through the simulation, for each time step, as each person moves in turn. Pathfinding arrays replace tasks and desks with traversable space, and people occupied cells with people IDs, such that other people’s paths avoid them.
* Social distancing arrays: social distancing arrays are modified pathfinding arrays, in which cells occupied by people are detected and the cells adjacent to them are also made untraversable (0). This means a social distancing person will leave 2m between themselves and others if possible (as one cell is 1m and a person is considered as a point in the cell center).
* Display arrays: display arrays are processed in the simulation script but initialised in the office class. They are RGB arrays, to show infected people (red), infectious people (orange), uninfected people (green), walls (black), tasks (grey) and desks (purple). They are exported as figures and displayed by the GUI.


The office class also reads the pathfinding arrays as each person moves, to detect when people ‘interact’ when they get within 2 meters of each other. Interactions are stored as a list of pairs of people and their distance to each other for a given frame in the simulation. This list is passed to the transmission class.

## Person Class

The person class is used to generate people with different attributes such as age, desk location, social distancing adherence, and initial infection status. Methods to assign tasks (I.e., location of task and how long it will take), find paths to those tasks and move along those paths are also defined here. The ‘pathfinding’ module is used here to find a route from each person’s current location to their task location, through a pathfinding array generated by the office class, using the A* algorithm which was found to be more efficient than Dijkstra’s algorithm.

## Transmission

The transmission code is run every frame of the simulation, its job is to determine who will become infected.  The inputs for this process include the office interactions for the frame, people (all person objects), and the ‘virality’ of the Covid-19 variant being modelled. Each interaction in the list contains two IDs corresponding to two individuals, and the distance between them.  To improve optimisation, the interactions list is filtered to remove any interactions where an infection is not possible.  Only interactions where one individual is infected and contagious, and the other is not infected are considered.  

* Specific mask adherence (the number of masks in the interaction, 2, 1, or 0).
* Distance between individuals (inverse square law factor).
* Virality (the contagiousness of the virus as set within the GUI / parameters file).

A probability of infection is calculated on a case-by-case basis for each interaction, and a random number used to determine the outcome.  Newly infected people are not contagious, but they are carriers of the virus.  When an infection has taken place the people class is updated to reflect infection state, and the ID of the person that infected them (for track and trace).

## Track and Trace

The track and trace process is performed once at the end of the simulation.  This is a terminal output, utilizing the Anytree module for building the tree structure shown in Figure 3.  The first stage is to find the IDs of those initially infected (who are contagious), and then to gather the individuals that they infected (using the infector ID within the person object).  The tree diagram is built by defining nodes, and their respective ‘parents’ and ‘children’ in the same format as a directory.  This tree is especially useful for visualising the effect of parameters on the spread of the virus (for example mask adherence).  

<img src="Report_graphics/track_trace.png" width=400 height=400 />

Figure 3:  Terminal output showing the IDs of infectors, and the individuals they have infected during the simulation.  Note the effect that mask wearing has on the number of those infected.

## Simulation

simulation.py brings the backend modules together to prepare outputs for the GUI or to be saved directly from the command line. Input parameters are checked that their datatype and values are as expected. Within this module, the people objects are instantiated and used to populate the office object. An RGB array is generated to display the office floorplan and people locations. The colour of each person denotes whether they are contagious, infected, or healthy. People are sequentially moved and any interactions between people are recorded. The transmission module is then used to determine if an infection has occurred and people colours are updated. Once the simulation has run for the number of iterations specified by the user, a list of RGB arrays containing the state of the office in each frame is returned to either be displayed in the GUI or saved to a ‘Plots’ file.

## Outputs

Simulation gifs can be saved from the GUI or command line. Examples of the best case scenario (left) where people are wearing masks and observing social distancing, and the worst case scenario (right) where people are not wearing masks or observing social distancing.

<tr>
    <td> <img src="Report_graphics/best_case.gif" alt="Drawing" style="width: 500px;"/> </td>
    <td> <img src="Report_graphics/worst_case.gif" style="width: 500px;"/> </td>
    </tr>

                          Best Case                                                          Worst Case